# Effectiveness - Delete a single point


In [ ]:
import sys
sys.path.append("..")

from sklearn import svm
import numpy as np

import dynashap

from examples.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize
)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
x_train, y_train, x_test, y_test, _ = preprocess_data('train_100p_3c.csv', 'test_100p_3c.csv')

mc_plus_sv_99 = dynashap.mc_shap(x_train[:99], y_train[:99], x_test, y_test, model, m=99*1000)
save_npy('mc_plus_sv_99.npy', mc_plus_sv_99)

mc_sv_99 = dynashap.mc_shap(x_train[:99], y_train[:99], x_test, y_test, model, m=99*100)
save_npy('mc_sv_99.npy', mc_plus_sv_99)

In [ ]:
# delta
init_sv = load_npy('mc_plus_sv_100.npy')

delta_shap = dynashap.DeltaShap(x_train, y_train, x_test, y_test, model,
                                init_sv)
delta_sv_99 = delta_shap.del_single_point(99, 100*(len(y_train)-1))

save_npy('delta_sv_99.npy', delta_sv_99)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_sv = load_npy('mc_plus_sv_100.npy')

# KNN
heur_shap = dynashap.HeurShap(x_train, y_train,
                              x_test, y_test,
                              model, init_sv,
                              params={'method': 'knn'})
heur_shap.prepare(params={'n_neighbors': 3})
knn_sv_99 = heur_shap.del_single_point(99)
save_npy('knn_sv_99.npy', knn_sv_99)

# for simi
model = svm.SVC(decision_function_shape='ovo')

idxs = [18, 56, 89]
svs = list()

for idx in idxs:
    sv = dynashap.mc_shap(np.delete(x_train, idx, axis=0),
                          np.delete(y_train, idx),
                          x_test, y_test,
                          model, 1000*99, 4)
    save_npy('mc_plus_sv_99_del'+str(idx)+'.npy', sv)
    svs.append(sv)

svs = np.asarray(svs)

save_npy('knn_plus_svs.npy', svs)

# KNN+
heur_shap = dynashap.HeurShap(x_train, y_train,
                              x_test, y_test,
                              model, init_sv,
                              params={'method': 'knn+'})

heur_shap.prepare(flags={'exact': False, 'train': False},
                  params={'n_neighbors': 3,
                          'simi_type': 'ed',
                          'f_shap': 'n*n',
                          'rela': ['poly', 1],
                          'train_idxs': [18, 56, 89],
                          'train_svs': load_npy('knn_plus_svs.npy')})

knn_plus_sv_99 = heur_shap.del_single_point(99)
save_npy('knn_plus_sv_99.npy', knn_plus_sv_99)

In [ ]:
# comparison
def comp(base_v, comp_sv, name):
    var = variance(base_v, normalize(base_v, comp_sv))
    print("variance of %s   \t : %.10f" % (name, var))
    

In [ ]:
comp(mc_plus_sv_99, load_npy('delta_sv_99.npy'), 'delta')
comp(mc_plus_sv_99, load_npy('knn_sv_99.npy'), 'knn')
comp(mc_plus_sv_99, load_npy('knn_plus_sv_99.npy'), 'knn+')
comp(mc_plus_sv_99, load_npy('mc_sv_99.npy'), 'mc')